In [1]:
import os
import sys
sys.path.append(os.path.join('..', '..', 'Quora_QP'))
import numpy as np
from preprocess.file_utils import deserialize, serialize
from itertools import chain
from preprocess.data_helpers import vectorize_sent

In [6]:
from gensim.models import KeyedVectors

In [2]:
data_dir = os.path.join('..', 'dataset')
train_x, train_y = deserialize(os.path.join(data_dir, 'train.bin'))
dev_x, dev_y = deserialize(os.path.join(data_dir, 'dev.bin'))
test_x = deserialize(os.path.join(data_dir, 'test.bin'))

loading 2 lines from ../dataset/train.bin
loading 2 lines from ../dataset/dev.bin
loading 2345796 lines from ../dataset/test.bin


In [3]:
word_train_set = set(chain.from_iterable(chain.from_iterable(train_x)))
word_dev_set = set(chain.from_iterable(chain.from_iterable(dev_x)))
word_test_set = set(chain.from_iterable(chain.from_iterable(test_x)))
word_set = word_train_set | word_dev_set | word_test_set

In [4]:
len(word_set)

141870

In [8]:
model = KeyedVectors.load_word2vec_format('../dataset/GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
word_embed_dict = {}
for word in word_set:
    if word in model.vocab:
        word_embed_dict[word] = model[word].tolist()

In [20]:
len(word_embed_dict)

58277

In [24]:
_PAD = '_PAD'
_NULL = '_NULL'
EMBEDDING_SIZE = 300
OOV_EMBED_SIZE = 5000


index2word = [_PAD, _NULL] + list(word_embed_dict.keys())
word2index = dict([(y, x) for (x, y) in enumerate(index2word)])
word_embeddings = [[0.0] * EMBEDDING_SIZE, [0.0] * EMBEDDING_SIZE]
for _, word in enumerate(index2word[2:]):
    word_embeddings.append(word_embed_dict[word])
for _ in range(OOV_EMBED_SIZE):
    word_embeddings.append(np.random.uniform(low=-0.1, high=0.1, size=[EMBEDDING_SIZE]).tolist())

In [25]:
serialize(word2index, os.path.join(data_dir, 'word2index_word2vec.bin'))
serialize(word_embeddings, os.path.join(data_dir, 'word_embeddings_word2vec.bin'))

saving 58279 lines to ../dataset/word2index_word2vec.bin
saving 63279 lines to ../dataset/word_embeddings_word2vec.bin
